## ANN - Regression

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasRegressor # pip install scikeras[tensorflow]
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV


In [ ]:
# Reading data 
import pandas as pd

from google.colab import files
file = files.upload()
import io
data = pd.read_csv(io.BytesIO(file['mart.csv']))

'''data = pd.read_csv('/content/mart.csv')
data.head()'''

Saving mart.csv to mart.csv


'data = pd.read_csv("D:/APU/CT108-3-3 - OCDS/Lab Sessions/Lab11 - Neural Netwrok/mart.csv")\ndata.head()'

In [ ]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [ ]:
data.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
data.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

In [ ]:
data['Item_Weight'].fillna((data['Item_Weight'].mean()), inplace=True)
data['Item_Visibility'] = data['Item_Visibility'].replace(0,np.mean(data['Item_Visibility']))
data['Outlet_Establishment_Year'] = 2013 - data['Outlet_Establishment_Year']

data['Outlet_Size'].fillna('Small',inplace=True)

In [ ]:
data = data.iloc [ : , 1: 12]
# creating dummy variables to convert categorical into numeric values
mylist = list(data.select_dtypes(include=['object']).columns)
dummies = pd.get_dummies(data[mylist], prefix= mylist)
data.drop(mylist, axis=1, inplace = True)
X = pd.concat([data,dummies], axis =1 )
y = X.iloc [ :, 4 ]
# drop the response variable
X = X.drop('Item_Outlet_Sales',1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
X_train.shape

(6818, 45)

In [ ]:
def baseline_model(optimizer='adam'):
    model = Sequential()
    model.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu', input_dim = 45)) # input layer
    model.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'linear')) # output layer
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
    return model

# Grid Search
epochs = [50, 100, 150]
batches = [16, 32, 64, 128]
optimizers = ['rmsprop', 'adam']

# Create hyperparameter options
hyperparameters = dict(optimizer = optimizers, epochs = epochs, batch_size = batches)
regressor = KerasRegressor(model = baseline_model) 
  
grid_search = GridSearchCV(estimator = regressor, param_grid = hyperparameters, cv = 5)

grid_search = grid_search.fit(X_train, y_train)

best_param = grid_search.best_params_
best_score = grid_search.best_score_

print(best_score)
print (best_param)

Streaming output truncated to the last 5000 lines.
43/43 [==============================] - 0s 3ms/step - loss: 1884226.5000 - mse: 1884226.5000
Epoch 62/100
43/43 [==============================] - 0s 4ms/step - loss: 1881712.2500 - mse: 1881712.2500
Epoch 63/100
43/43 [==============================] - 0s 3ms/step - loss: 1877625.8750 - mse: 1877625.8750
Epoch 64/100
43/43 [==============================] - 0s 2ms/step - loss: 1874670.6250 - mse: 1874670.6250
Epoch 65/100
43/43 [==============================] - 0s 3ms/step - loss: 1871542.2500 - mse: 1871542.2500
Epoch 66/100
43/43 [==============================] - 0s 3ms/step - loss: 1868673.6250 - mse: 1868673.6250
Epoch 67/100
43/43 [==============================] - 0s 3ms/step - loss: 1865393.2500 - mse: 1865393.2500
Epoch 68/100
43/43 [==============================] - 0s 3ms/step - loss: 1861808.5000 - mse: 1861808.5000
Epoch 69/100
43/43 [==============================] - 0s 4ms/step - loss: 1858509.8750 - mse: 1858509.8750

In [ ]:
model = Sequential()
model.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu', input_dim = 45)) # input layer
model.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'relu'))
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'linear')) # output layer
model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mse'])

model.fit(X_train, y_train, validation_data = (X_test, y_test), batch_size = 16, epochs = 150, verbose = 1)


Epoch 1/150
427/427 [==============================] - 2s 2ms/step - loss: 7004985.5000 - mse: 7004985.5000 - val_loss: 5814416.5000 - val_mse: 5814416.5000
Epoch 2/150
427/427 [==============================] - 1s 2ms/step - loss: 3228428.5000 - mse: 3228428.5000 - val_loss: 2035165.1250 - val_mse: 2035165.1250
Epoch 3/150
427/427 [==============================] - 1s 2ms/step - loss: 1971386.5000 - mse: 1971386.5000 - val_loss: 2006785.2500 - val_mse: 2006785.2500
Epoch 4/150
427/427 [==============================] - 1s 2ms/step - loss: 1955705.0000 - mse: 1955705.0000 - val_loss: 1995449.2500 - val_mse: 1995449.2500
Epoch 5/150
427/427 [==============================] - 1s 2ms/step - loss: 1938467.6250 - mse: 1938467.6250 - val_loss: 1974834.8750 - val_mse: 1974834.8750
Epoch 6/150
427/427 [==============================] - 1s 2ms/step - loss: 1924178.5000 - mse: 1924178.5000 - val_loss: 1969631.2500 - val_mse: 1969631.2500
Epoch 7/150
427/427 [==============================] - 1s 

In [ ]:
y_predict = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_predict, y_test)

1181836.3101409052